<a href="https://colab.research.google.com/github/KaifAhmad1/AB-Testing/blob/master/NLP_Engineer_Assignment_Mohd_Kaif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU transformers
!pip install -qU peft
!pip install -qU Optimum
!pip install -qU einops
!pip install -qU datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments,
    pipeline, logging
)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
file_path = '/content/drive/MyDrive/Network-QA-Dataset.csv'

In [ ]:
data = pd.read_csv(file_path)

In [ ]:
data

,Questions,Answers,Context Info,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,What is the scope of the technical specificati...,The scope of the technical specification is de...,"The technical specification, titled ""3GPP TS 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Where can specifications and reports for the i...,Specifications and reports for the implementat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,What are the different restoration indicators ...,The document discusses various restoration ind...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What procedures are outlined for the restorati...,Procedures for the restoration of data in the ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,In which section can information about the res...,Information about the restoration of data in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,What actions does the UE take if the security ...,If the security check is successful in step 7:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,What actions does the UE take if the security ...,If the security check is not successful in ste...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,What does the UE do if the UDM has requested a...,If the UDM has requested an acknowledgment and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,What operations does the AMF perform if an SOR...,The AMF uses the Nudm_SDM_Info service operati...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
network_data = data[['Questions','Answers', 'Context Info']]

In [ ]:
network_data

,Questions,Answers,Context Info
0,What is the scope of the technical specificati...,The scope of the technical specification is de...,"The technical specification, titled ""3GPP TS 2..."
1,Where can specifications and reports for the i...,Specifications and reports for the implementat...,NaN
2,What are the different restoration indicators ...,The document discusses various restoration ind...,NaN
3,What procedures are outlined for the restorati...,Procedures for the restoration of data in the ...,NaN
4,In which section can information about the res...,Information about the restoration of data in ...,NaN
...,...,...,...
599,What actions does the UE take if the security ...,If the security check is successful in step 7:...,NaN
600,What actions does the UE take if the security ...,If the security check is not successful in ste...,NaN
601,What does the UE do if the UDM has requested a...,If the UDM has requested an acknowledgment and...,NaN
602,What operations does the AMF perform if an SOR...,The AMF uses the Nudm_SDM_Info service operati...,NaN


In [ ]:
network_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Questions     604 non-null    object
 1   Answers       604 non-null    object
 2   Context Info  32 non-null     object
dtypes: object(3)
memory usage: 14.3+ KB


In [ ]:
print(network_data.columns.tolist())

['Questions', 'Answers', 'Context Info']


**Data Preprocessing**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
def process_data_sample(example):
    question = example['Questions']
    answer = example['Answers']
    context_info = example['Context Info']

    # Convert potential NaN values to empty string
    question = str(question)
    answer = str(answer)
    context_info = str(context_info) if pd.notna(context_info) else ""

    processed_example = "\nYou are a Question Answering System who helps with user queries chatbot who always responds in the style of a professional.\n\n" + question + "\n\n" + answer + "\n\n" + context_info
    return processed_example

In [ ]:
network_data['text'] = network_data[['Questions', 'Answers', 'Context Info']].apply(lambda x: process_data_sample(x), axis=1)

<ipython-input-12-6fa798a8e3a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  network_data['text'] = network_data[['Questions', 'Answers', 'Context Info']].apply(lambda x: process_data_sample(x), axis=1)


In [ ]:
print(network_data[['Questions', 'Answers', 'Context Info']].isnull().sum())

Questions         0
Answers           0
Context Info    572
dtype: int64


In [ ]:
print(network_data.iloc[5]['text'])


You are a Question Answering System who helps with user queries chatbot who always responds in the style of a professional.

What is discussed in Section 16.2 regarding the SGW?

Section 16.2 discusses partial failure handling at the SGW, outlining general procedures during PDN Connection Establishment and actions during SGW Partial Failure.




In [ ]:
from datasets import Dataset
# Convert the Dataframe to Huggingface Dataset
finetuning_dataset = Dataset.from_pandas(network_data[['text']])
finetuning_dataset

Dataset({
    features: ['text'],
    num_rows: 604
})

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
# Load Tokenizer
model_name ='mistralai/Mistral-7B-v0.1'

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
# Add a padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
# Configuration for quantization
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                              quantization_config=bnb_config,
                                              device_map="auto",
                                              use_cache=False,
                                              trust_remote_code=True
                                              )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
print(model)

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

In [ ]:
# Update Model Configurations
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA Config:
peft_config = LoraConfig(
                            r=16,
                            lora_alpha=16,
                            lora_dropout=0.05,
                            bias='none',
                            task_type="CAUSAL_LM",
                            target_modules=["q_proj","v_proj"]
                        )

In [ ]:
print(model)

In [ ]:
# Training Arguments:
training_arguments = TrainingArguments(
                                           output_dir='Mistral-Network-QnA-System',
                                           per_device_train_batch_size=8,
                                           gradient_accumulation_steps=1,
                                           optim='paged_adamw_32bit',
                                           learning_rate=2e-4,
                                           lr_scheduler_type='cosine',
                                           save_strategy='epoch',
                                           logging_steps=50,
                                           num_train_epochs=1,
                                           max_steps=250,
                                           fp16=True,
                                           push_to_hub=False,
                                      )

In [ ]:
# SFT Trainer Arguments:
trainer = SFTTrainer(
                        model=model,
                        train_dataset=finetuning_dataset,
                        peft_config=peft_config,
                        dataset_text_field='text',
                        args=training_arguments,
                        tokenizer=tokenizer,
                        packing=False,
                        max_seq_length=1024
                    )

In [ ]:
trainer.train()